In [11]:
import pandas as pd
import numpy as np 
import datetime
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
import optuna.integration.lightgbm as lgb_o

In [15]:
results = pd.read_pickle('results.pickle')

In [16]:
class HorseResults:
    @staticmethod
    def scrape(horse_id_list):
        
        #horse_idをkeyにしてデータフレーム型を格納
        horse_results = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = 'https://db.netkeiba.com/horse/' + horse_id
                df = pd.read_html(url)[3]
                #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                if df.columns[0]=='受賞歴':
                    df = pd.read_html(url)[4]          
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df
                time.sleep(0.5)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
                
        #pd.DataFrame型にして一つのデータにまとめる        
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])
        
        return horse_results_df

In [17]:
horse_id_list = results['horse_id'].unique()
horse_results = HorseResults.scrape(horse_id_list)
horse_results

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2017105318,2020/12/26,5中山7,晴,9,立志賞(1勝クラス),NaN,18,6,12,3.5,...,1.9,**,1-1-1-1,37.5-36.8,38.7,570(+18),NaN,NaN,ルトロヴァイユ,NaN
2017105318,2020/08/02,1札幌4,晴,7,3歳以上1勝クラス,NaN,8,8,8,1.5,...,0.2,**,7-6-4-4,36.6-34.7,34.5,552(+28),NaN,NaN,シャムロックヒル,110.0
2017105318,2019/08/31,2札幌5,曇,11,札幌2歳S(G3),NaN,12,7,10,1.8,...,0.6,**,10-7-7-2,36.0-37.3,37.3,524(+6),NaN,NaN,ブラックホール,NaN
2017105318,2019/07/27,1札幌1,曇,1,2歳未勝利,NaN,9,1,1,1.4,...,-1.8,**,1-1-1-1,35.9-36.5,36.5,518(-16),NaN,NaN,(プントファイヤー),500.0
2017105318,2019/06/09,3東京4,曇,5,2歳新馬,NaN,11,4,4,4.8,...,0.5,**,8-8-2,38.7-33.5,34.0,534(0),NaN,NaN,ワーケア,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016105550,2020/04/28,門別,曇,4,3歳以上C4ー3,NaN,8,1,1,6.4,...,0.8,**,2-3,0.0-40.0,40.5,500(-4),NaN,NaN,ファイトフラッシュ,NaN
2016105550,2020/04/15,門別,曇,6,3歳以上C4ー2,NaN,12,3,3,5.0,...,1.5,**,5-5,0.0-39.1,39.3,504(0),NaN,NaN,エスシーヴァローナ,NaN
2016105550,2019/07/27,1札幌1,晴,6,3歳未勝利,NaN,8,1,1,5.4,...,NaN,**,4-4-8-8,30.2-40.0,NaN,504(+8),NaN,NaN,ダイヤレイジング,NaN
2016105550,2019/05/18,1新潟7,晴,7,3歳未勝利,NaN,13,7,10,12.6,...,2.1,**,3-3-2-2,35.7-35.6,37.6,496(0),NaN,NaN,パルティアーモ,NaN


In [18]:
horse_results.to_pickle('horse_results.pickle')